## Do not forget to click Runtime -> Change Runtime type -> Hardware accelerator ->GPU

<a href="https://colab.research.google.com/github/yandexdataschool/MLatImperial2020/blob/master/06_lab/baseline_kaggle_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

As before - mount drive, download data

In [0]:
DATA_PREFIX = "/content/gdrive/My Drive/kaggle_2/"

In [0]:
import os
os.makedirs(DATA_PREFIX)

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
!mkdir /root/.kaggle
!cp /content/gdrive/My\ Drive/kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!ls -l /root/.kaggle

In [8]:
!kaggle competitions download --force -c mlimperial2020-2 -p '{DATA_PREFIX}'

!unzip '{DATA_PREFIX}/kaggle_train.h5.zip' -d '{DATA_PREFIX}'
!chmod +rw '{DATA_PREFIX}/kaggle_train.h5'
!rm '{DATA_PREFIX}/kaggle_train.h5.zip'


!unzip '{DATA_PREFIX}/kaggle_test.h5.zip' -d '{DATA_PREFIX}'
!chmod +rw '{DATA_PREFIX}/kaggle_test.h5'
!rm '{DATA_PREFIX}/kaggle_test.h5.zip'

 98% 321M/329M [00:03<00:00, 87.8MB/s]
100% 329M/329M [00:03<00:00, 110MB/s] 
100% 985M/988M [00:24<00:00, 128MB/s]
100% 988M/988M [00:24<00:00, 42.3MB/s]
unzip:  cannot find or open /content/gdrive/My Drive/kaggle_2//kaggle_train.h5.zip, /content/gdrive/My Drive/kaggle_2//kaggle_train.h5.zip.zip or /content/gdrive/My Drive/kaggle_2//kaggle_train.h5.zip.ZIP.
rm: cannot remove '/content/gdrive/My Drive/kaggle_2//kaggle_train.h5.zip': No such file or directory
Archive:  /content/gdrive/My Drive/kaggle_2//kaggle_test.h5.zip
replace /content/gdrive/My Drive/kaggle_2/kaggle_test.h5? [y]es, [n]o, [A]ll, [N]one, [r]ename: т
error:  invalid response [т]
replace /content/gdrive/My Drive/kaggle_2/kaggle_test.h5? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [0]:
!ls '{DATA_PREFIX}'

In [0]:
# Create folder to save NN snapshots during training
os.makedirs(os.path.join("/content/gdrive/My Drive/kaggle_2", "nn_snapshots"))

# !Link to challenge!

# https://www.kaggle.com/t/3814f61079e947cda51a13cb9a7b582a

## Metric - ROC AUC

Your task is to try as many techniques that you have learned this week as possible.

The outcome of your work should be a small table with results, i.e 
Method - parameters tuned with CV - score + features created on top of exiting ones. The table should be accompanied by a small report of your workflow and reasoning. Also, you need to send the code.

The archive with the files should be sent to mlicl-2020@yandex.ru with the topic: **Surname_name_kaggle_2**

The total amount of points is 10. You will get additional points based on your final ranking.

** 1 Point **

Try different layers, such as Dropout, BN, different poolings, convs. Do all of them work? Why?

** 1 Point **

Try different activations, i.e. relu, sigmoid, tanh etc. Do all of them work? Why?

** 1 Point **

Try different optimisers, i.e. SGD, Adam, Adamax, rmsprop ... Which you find the best? Which converges faster?

** 1 Point **

Try different depth of network, different order of layers. What do you observe?

** 3 Points **

Augment(rotation, jitter, reflection ...) you data using symmetries in the data.

** 1 Points **

Change optimisation loop/optimiser to set up decay of learning rate by the scheme of your choice, abort trainin by stopping criteria of your choice.

** 2 Points **

Use pretrained nets / part of nest. Does this technique improve the score?

** Bonus **

Beat medium baseline - + 2 bonus points.




# Util functions to preprocess and work with data

In [0]:
def read_data(data, is_train=True, start_ind=0, end_ind=0):
    layer_hcal = np.expand_dims(data['all_events']['histHCAL'][start_ind : end_ind], -1).astype(np.float32)
    layer_em = np.expand_dims(data['all_events']['histEM'][start_ind : end_ind], -1).astype(np.float32)
    layer_track = np.expand_dims(data['all_events']['histtrack'][start_ind : end_ind], -1).astype(np.float32)
    
    hit_map = np.concatenate((layer_hcal, layer_em, layer_track), axis=-1).astype(np.float32)
    hit_map = np.rollaxis(hit_map, 3, 1)
    hit_map = (hit_map - hit_map.mean(axis=0, keepdims=True)) / hit_map.std(axis=0, keepdims=True)
    answers = None
    if is_train:
        answers = np.expand_dims(data['all_events']['y'][start_ind : end_ind], -1)
    return hit_map, answers
  
def save_data_in_chunks(data, chunk_size):
    for index, step in enumerate(range(0, len(data['all_events']['histHCAL']), chunk_size)):
        X, y = read_data(train, True, step, step + chunk_size)
        X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.9, random_state=42)
        np.save("X_train_{}".format(index) , X_train)
        np.save("y_train_{}".format(index) , y_train)        
        np.save("X_val_{}".format(index) , X_val)
        np.save("y_val_{}".format(index) , y_val)
        del X, y, X_train, X_val, y_train, y_val
        gc.collect()
        print("Done:{}".format(index))  

### import standard packages

In [0]:
import gc
import numpy as np
import pickle
import os
import matplotlib.pyplot as plt
%matplotlib inline
import scipy
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

## import torch and h5 data format parser

In [0]:
import h5py
import torch

### Create small batches of train data locally(will be deleted after restrat of the session)

In [0]:
train = h5py.File(os.path.join(DATA_PREFIX, "kaggle_train.h5"), 'r')
save_data_in_chunks(train, 50000)

Event example

In [0]:
X_example = np.load("X_train_0.npy")

In [0]:
f, ax = plt.subplots(1,3,figsize=(14,6))
for i in range(3):
    ax[i].imshow(X_example[100,i,:,:], cmap="Reds")

In [0]:
del X_example
gc.collect()

Load validation data to memory

In [0]:
# This variable tells, how many file pieces of validation data we want to consider
N_DATA_SPLITS = 1

In [0]:
X_val = np.concatenate([np.load("X_val_{}.npy".format(i)) for i in range(N_DATA_SPLITS)])
y_val = np.concatenate([np.load("y_val_{}.npy".format(i)) for i in range(N_DATA_SPLITS)])

## The next steps are exactly the same as on  Monday

In [0]:
from torch import nn
import torch.nn.functional as F

class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

### This is dummy example of CNN, look what you can change here

In [0]:
device = torch.device("cuda", 0)

In [0]:
model = torch.nn.Sequential()
model.add_module("maxpool_1", torch.nn.MaxPool2d(kernel_size=2))
model.add_module('conv_1', nn.Conv2d(3, 32, kernel_size=(5,5), stride=1, padding=0))
model.add_module("maxpool_2", torch.nn.MaxPool2d(kernel_size=2))
model.add_module("relu_1", torch.nn.ReLU())

model.add_module("flat", Flatten())

model.add_module("fc1", torch.nn.Linear(6272, 128))
model.add_module("relu_2", torch.nn.ReLU())
model.add_module("fc2", torch.nn.Linear(128, 1))
model.add_module("sigmoid", torch.nn.Sigmoid())

model.to(device)

### Training on minibatches

Just like before, we train our model on small random minibatches of data with adaptive optimization method of your choice.

In [0]:
# An auxilary function that returns mini-batches for neural network training
from tqdm import trange
def iterate_minibatches(X, y, batchsize, shuffle=False):
    indices = np.arange(len(X))
    if shuffle: 
        indices = np.random.permutation(indices)
    for start in trange(0, len(indices), batchsize):
        ix = indices[start: start + batchsize]
        yield X[ix], y[ix]

### Choose you optimiser

In [0]:
opt = torch.optim.SGD(model.parameters(), lr=0.01)

## And set up batch_size and number of epochs

In [0]:
import time
#from pandas import ewma
from IPython import display

num_epochs = 2 #amount of passes through the data
batch_size = 1024 #number of samples processed at each function call
auc_history = []

number_of_chunks = 1 #number of initial data splits to process
best_score = 0
best_epoch = 0

In [0]:
for epoch in range(num_epochs):
    # In each epoch, we do a full pass over the training data:
    
    train_err = train_acc = 0
    train_batches = 0
    start_time = time.time()
    
    for step in range(number_of_chunks):
        X_train, y_train = np.load("X_train_{}.npy".format(step)), np.load("y_train_{}.npy".format(step))
        train_batches += np.ceil(len(X_train) / batch_size).astype(int)
        # This is you have see already - traning loop
        model.train(True) # enable dropout / batch_norm training behavior
        for X_batch, y_batch in iterate_minibatches(X_train, y_train, batch_size, shuffle=True):
            X_batch = torch.FloatTensor(X_batch).to(device)
            y_batch = torch.FloatTensor(y_batch).to(device)

            y_predicted = model(X_batch)
            loss = torch.nn.functional.binary_cross_entropy(y_predicted, y_batch).mean()

            loss.backward()
            opt.step()
            opt.zero_grad()

            train_err += loss.data.cpu().numpy()
            train_acc += torch.eq(torch.round(y_predicted), y_batch).data.cpu().numpy().mean()

    # And a full pass over the validation data:
    y_pred = []

    model.train(False)
    for X_batch, y_batch in iterate_minibatches(X_val, y_val, batch_size, shuffle=False):
        X_batch = torch.FloatTensor(X_batch).to(device)
        y_pred.extend(model(X_batch).data.cpu().numpy())

    y_pred = np.asarray(y_pred)
    # Save the metrics values   
    val_acc = accuracy_score(y_val, y_pred > 0.5)
    val_roc_auc = roc_auc_score(y_val, y_pred)
    auc_history.append(val_roc_auc)


    # Visualize
    display.clear_output(wait=True)
    plt.figure(figsize=(8, 6))
    plt.title("Validation AUC")
    plt.xlabel("#iteration")
    plt.ylabel("AUC")
    plt.plot(auc_history, 'b',label='val auc')
    plt.legend(loc='best')
    plt.grid()
    plt.show()

    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, num_epochs, time.time() - start_time))

    print("  training loss (in-iteration):\t\t{:.6f}".format(train_err / train_batches))
    print("  train accuracy:\t\t{:.2f} %".format(train_acc / train_batches * 100))
    print("  validation accuracy:\t\t{:.2f} %".format(val_acc * 100))
    print("  validation roc_auc:\t\t{:.2f} %".format(val_roc_auc * 100))
    
    
    if auc_history[-1] > best_score:
        best_score = auc_history[-1]
        best_epoch = epoch
        with open(os.path.join("/content/gdrive/My Drive/kaggle_2", "nn_snapshots", "best.pt"), 'wb') as f:
            torch.save(model, f)    

# Read test data, feed it to neural network, and save the output in kaggle fromat.

In [0]:
chunk_size = 10000
test = h5py.File(os.path.join(DATA_PREFIX, "kaggle_test.h5"), 'r')

y_ans = []
for index, step in enumerate(range(0, len(test['all_events']['histHCAL']), chunk_size)):
    X, _ = read_data(test, False, step, step + chunk_size)
    y_ans.extend(model(torch.FloatTensor(X).to(device)).detach().cpu().numpy())
    del X
    gc.collect()
    print("Done:{}".format(index))
    
y_ans = np.array(y_ans)

Saving you results to file.

In [0]:
import pandas as pd
from IPython.display import FileLink

def save_results(filename, y_ans):
    answer_dataframe = pd.DataFrame(columns=["ID", "ans"])
    answer_dataframe['ID'] = range(0,len(y_ans))
    answer_dataframe['ans'] = y_ans
    answer_dataframe.to_csv(os.path.join(DATA_PREFIX, '{}'.format(filename)), index=False)
    return FileLink(os.path.join(DATA_PREFIX, '{}'.format(filename)))

In [0]:
save_results("baseline.csv", y_ans)

In [0]:
!kaggle competitions submit -c mlimperial2020-2 -f "{DATA_PREFIX}/baseline.csv" -m "Message"